In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings
import pylab
from datetime import datetime

In [2]:
datefile = datetime.now().strftime("%Y_%m_%d")

## KTTR

In [3]:
KTTR = pd.read_csv('KTTR3_20102016.csv', sep=';', encoding='utf-8', 
    usecols=['DRWH Patient ID', 'IPP', 'Date document', 'POIDS', 'SAT_texte', 'SAT', 'SAT.1', 
             'SVO2', 'SVO2.1', 'SVO2.2', 'SVO2.3', 'SVO2.4', 'SVO2.5', 'SVO2.6', 'SVO2.7', 
             'SVO2.8', 'PAPS', 'PAPS.1', 'PAPS.2', 'PAPM', 'PAPM.1', 'PAPM.2','PTDVG', 
             'PTDVG.1', 'PTDVG.2', 'RVP', 'RVPi_texte', 'QPQS', 'QPQS_texte', 'Debit Indexe'])

In [4]:
KTTR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5320 entries, 0 to 5319
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DRWH Patient ID  5320 non-null   int64  
 1   IPP              5320 non-null   object 
 2   Date document    5317 non-null   object 
 3   POIDS            4521 non-null   object 
 4   SAT_texte        582 non-null    float64
 5   SAT              37 non-null     float64
 6   SAT.1            3224 non-null   object 
 7   SVO2             49 non-null     float64
 8   SVO2.1           145 non-null    float64
 9   SVO2.2           912 non-null    float64
 10  SVO2.3           274 non-null    float64
 11  SVO2.4           793 non-null    float64
 12  SVO2.5           55 non-null     float64
 13  SVO2.6           143 non-null    float64
 14  SVO2.7           352 non-null    object 
 15  SVO2.8           64 non-null     float64
 16  PAPS             2008 non-null   float64
 17  PAPS.1        

In [5]:
# les sat notées en pourcentages dans SAT.1 sortent ,dd à l'extraction
KTTR['SAT.1'] = KTTR['SAT.1'].str.replace(',', '', regex=True)

In [6]:
# Enlever les Unités Wood & cie 
KTTR["RVPi_texte"].replace({"2:": "2", "2+": "2", "1U": "1", "5W": "5"}, inplace=True)

In [7]:
# 1/2 =0.5
KTTR["QPQS_texte"].replace({"2:": "2", "2/": "2", "1/1": "1", "2/1": "2", "3/1": "3", "2+": "2"}, inplace=True)

In [8]:
# Remplacer les , par des .
KTTR['POIDS'] = KTTR['POIDS'].str.replace(',', '.', regex=True)
KTTR['RVP'] = KTTR['RVP'].str.replace(',', '.', regex=True)
KTTR['RVPi_texte'] = KTTR['RVPi_texte'].str.replace(',', '.', regex=True)
KTTR['QPQS'] = KTTR['QPQS'].str.replace(',', '.', regex=True)
KTTR['QPQS_texte'] = KTTR['QPQS_texte'].str.replace(',', '.', regex=True)
KTTR['SVO2.7'] = KTTR['SVO2.7'].str.replace(',', '.', regex=True)
KTTR['Debit Indexe'] = KTTR['Debit Indexe'].str.replace(',', '.', regex=True)

In [9]:
# cast data in expected types 
KTTR['Date document'] = pd.to_datetime(KTTR['Date document'], dayfirst=True, errors='coerce')
KTTR['IPP'] = pd.to_numeric(KTTR['IPP'], errors='coerce') # Convert IPP to int64 replace not numeric (KTTR_5824, ASTRAIA...) by NaN
KTTR = KTTR.astype(dtype={
    'POIDS': float, 'SAT.1': float, 'SVO2.7': float, 
    'RVP': float, 'RVPi_texte': float, 'QPQS': float, 
    'QPQS_texte': float, 'Debit Indexe': float, })

In [10]:
# drop null values 
KTTR = KTTR[KTTR['Date document'].notnull()]
KTTR = KTTR[(2009 < KTTR["Date document"].dt.year) & (KTTR["Date document"].dt.year < 2017)]

In [11]:
# Filter abherent values 
l_h_bounds = {
    'POIDS': [0, 200], 
    'SAT' : [20, 100], 'SAT.1': [20, 100], 'SAT_texte': [20, 100], 
    "RVPi_texte": [0, 100], "RVP": [0, 100], 
    'QPQS_texte': [0, 10], 'QPQS': [0, 10], 'Debit Indexe': [0, 10], 
    "SVO2": [0, 100], "SVO2.1": [0, 100], "SVO2.2": [0, 100], 
    "SVO2.3": [0, 100], "SVO2.4": [0, 100], "SVO2.5": [0, 100], 
    "SVO2.6": [0, 100], "SVO2.7": [0, 100], "SVO2.8": [0, 100], 
    'PAPS': [0, 200], 'PAPS.2': [0, 200], 'PAPS.1': [0, 200], 
    'PAPM': [0, 200], 'PAPM.2': [0, 200], 'PAPM.1': [0, 200], 
    'PTDVG': [0, 100], 'PTDVG.2': [0, 100], 'PTDVG.1': [0, 100], 
}

for var, bounds in l_h_bounds.items():
    KTTR.loc[(KTTR[var] < bounds[0]) | (bounds[1] < KTTR[var]), var] = np.nan
    print(var, "minmax (", KTTR[var].min(), ", ", KTTR[var].max(), ")")

POIDS minmax ( 1.0 ,  114.0 )
SAT minmax ( 74.0 ,  83.0 )
SAT.1 minmax ( 70.0 ,  100.0 )
SAT_texte minmax ( 70.0 ,  100.0 )
RVPi_texte minmax ( 0.8 ,  59.0 )
RVP minmax ( nan ,  nan )
QPQS_texte minmax ( 0.6 ,  10.0 )
QPQS minmax ( 1.15384615384615 ,  1.5 )
Debit Indexe minmax ( 1.2 ,  8.8 )
SVO2 minmax ( 66.0 ,  79.0 )
SVO2.1 minmax ( 59.0 ,  88.0 )
SVO2.2 minmax ( 21.0 ,  98.0 )
SVO2.3 minmax ( 30.0 ,  95.0 )
SVO2.4 minmax ( 23.0 ,  100.0 )
SVO2.5 minmax ( 10.0 ,  98.0 )
SVO2.6 minmax ( 10.0 ,  98.0 )
SVO2.7 minmax ( 27.0 ,  96.0 )
SVO2.8 minmax ( 38.0 ,  86.0 )
PAPS minmax ( 8.0 ,  185.0 )
PAPS.2 minmax ( 7.0 ,  154.0 )
PAPS.1 minmax ( 6.0 ,  161.0 )
PAPM minmax ( 4.0 ,  125.0 )
PAPM.2 minmax ( 6.0 ,  101.0 )
PAPM.1 minmax ( 3.0 ,  105.0 )
PTDVG minmax ( 2.0 ,  35.0 )
PTDVG.2 minmax ( 9.0 ,  26.0 )
PTDVG.1 minmax ( 5.0 ,  91.0 )


In [12]:
KTTR['SAT'] = KTTR[['SAT_texte', 'SAT', 'SAT.1']].mean(axis=1)
KTTR.drop(columns=['SAT_texte', 'SAT.1'], inplace=True)
KTTR['RVP'] = KTTR[['RVPi_texte', 'RVP']].mean(axis=1)
KTTR.drop(columns=['RVPi_texte'], inplace=True)
KTTR['QPQS'] = KTTR[['QPQS', 'QPQS_texte', 'Debit Indexe']].mean(axis=1)
KTTR.drop(columns=['QPQS_texte', 'Debit Indexe'], inplace=True)
KTTR['SVO2'] = KTTR[
    ['SVO2', 'SVO2.1', 'SVO2.2', 'SVO2.3', 'SVO2.4', 'SVO2.5','SVO2.6', 'SVO2.7', 'SVO2.8']
].mean(axis=1)
KTTR.drop(columns=['SVO2.1', 'SVO2.2', 'SVO2.3', 'SVO2.4', 'SVO2.5','SVO2.6', 'SVO2.7', 'SVO2.8'], inplace=True)
KTTR['PAPS'] = KTTR[['PAPS', 'PAPS.1', 'PAPS.2']].mean(axis=1)
KTTR.drop(columns=['PAPS.1', 'PAPS.2'], inplace=True)
KTTR['PAPM'] = KTTR[['PAPM', 'PAPM.1', 'PAPM.2']].mean(axis=1)
KTTR.drop(columns=['PAPM.1', 'PAPM.2'], inplace=True)
KTTR['PTDVG'] = KTTR[['PTDVG', 'PTDVG.1', 'PTDVG.2']].mean(axis=1)
KTTR.drop(columns=['PTDVG.1', 'PTDVG.2'], inplace=True)

In [13]:
# keep only averaged columns 
#KTTR = df[['DRWH Patient ID', 'IPP', 'Date document', 'POIDS', 'PTDVG_avg', 'SAT_avg',
#            'PAPS_avg', 'PAPM_avg', 'RVP_avg', 'QPQS_avg', 'SVO2_avg']].copy()
KTTR = KTTR.rename(columns={"DRWH Patient ID": "DRWHID_Num",})
KTTR['DRWHID'] = 'DRWH-' + KTTR['DRWHID_Num'].astype(str)
KTTR.sample(5)

,DRWHID_Num,IPP,Date document,POIDS,SAT,SVO2,PAPS,PAPM,PTDVG,RVP,QPQS,DRWHID
3088,78602,8.004194e+09,2015-10-05,13.15,96.0,NaN,NaN,NaN,NaN,NaN,NaN,DRWH-78602
2330,284062,8.003816e+09,2014-08-11,38.00,100.0,67.0,20.0,13.0,NaN,NaN,NaN,DRWH-284062
235,146938,1.810075e+09,2011-02-07,5.30,77.0,NaN,18.0,14.0,NaN,NaN,NaN,DRWH-146938
355,219812,8.004083e+09,2013-05-14,9.06,95.0,NaN,20.0,13.0,NaN,NaN,NaN,DRWH-219812
4493,282762,8.004287e+09,2014-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DRWH-282762


In [14]:
print('Nombre de patients inclus (dans KTTR):', KTTR['DRWHID_Num'].nunique())

KTTR.info()

Nombre de patients inclus (dans KTTR): 3082
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4136 entries, 0 to 5188
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   DRWHID_Num     4136 non-null   int64         
 1   IPP            4027 non-null   float64       
 2   Date document  4136 non-null   datetime64[ns]
 3   POIDS          3458 non-null   float64       
 4   SAT            2583 non-null   float64       
 5   SVO2           754 non-null    float64       
 6   PAPS           1934 non-null   float64       
 7   PAPM           2146 non-null   float64       
 8   PTDVG          246 non-null    float64       
 9   RVP            230 non-null    float64       
 10  QPQS           204 non-null    float64       
 11  DRWHID         4136 non-null   object        
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 420.1+ KB


## Type of procedure

In [15]:
TDK = pd.read_csv('tdkflair.csv', 
                sep=';', encoding='utf-8',
                parse_dates=['DATE_DE_KT'], dayfirst=True,
                usecols=['DWH_ID', 'tdk', 'DATE_DE_KT', 'IPP_DWH'])

TDK = TDK.rename(columns={'IPP_DWH': 'IPP', 'tdk': 'TDK', 'DWH_ID': 'DRWHID_Num'})

TDK.head(5)

TDK.info()

print('Nombre de patients distincts dans tdkflair:')
print(TDK['DRWHID_Num'].nunique())

# Merge KTTR & TDK on IPP & DATE (left-join on KTTR)
KTTR_TDK = pd.merge(KTTR,
               TDK,
               left_on=['DRWHID_Num', 'Date document'],
               right_on=['DRWHID_Num', 'DATE_DE_KT'],
               how='left'
             )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13326 entries, 0 to 13325
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   IPP         13325 non-null  float64       
 1   DRWHID_Num  13326 non-null  int64         
 2   DATE_DE_KT  13306 non-null  datetime64[ns]
 3   TDK         13326 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 416.6+ KB
Nombre de patients distincts dans tdkflair:
9208


In [16]:
KTTR_TDK[['TDK', 'TDK_proba']] = KTTR_TDK['TDK'].str.split(' ', 1, expand=True)

KTTR_TDK['TDK'] = KTTR_TDK['TDK'].astype(float)

KTTR_TDK['TDK'] = KTTR_TDK['TDK'].replace({1:'Autre', 
                                 2: 'BOM_hors_TPC',
                                 3: 'BOM_TPC',
                                 4: 'DX',
                                 7: 'Dilatation_AS',
                                 8: 'Dilatation_AP',
                                 9: 'Dilatation_COA',
                                 10: 'Drainage_PC',
                                 11: 'Extraction_CE',
                                 12: 'Fermeture_CIA_PFO',
                                 14: 'Fermeture_CAP',
                                 15: 'Fermeture_COLAT',
                                 17: 'Fermeture_Fistule_Coronaire',
                                 19: 'Melody',
                                 20: 'Pas_un_KT',
                                 22: 'Rashkind',
                                 26: 'Valvuloplastie'})

In [17]:
# Remove les Pas_un_KT

Pas_un_KT = KTTR_TDK[(KTTR_TDK['TDK'] == 'Pas_un_KT')].index
KTTR_TDK.drop(Pas_un_KT, inplace=True)

In [18]:
KTTR_TDK = KTTR_TDK.sort_values(['Date document']).drop_duplicates(subset='DRWHID_Num')

retenus = KTTR_TDK['DRWHID_Num'].nunique()

print('Nombre de patients inclus:', retenus)

Nombre de patients inclus: 2982


In [19]:
KTTR_TDK.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2982 entries, 1885 to 3146
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   DRWHID_Num     2982 non-null   int64         
 1   IPP_x          2890 non-null   float64       
 2   Date document  2982 non-null   datetime64[ns]
 3   POIDS          2538 non-null   float64       
 4   SAT            1869 non-null   float64       
 5   SVO2           503 non-null    float64       
 6   PAPS           1415 non-null   float64       
 7   PAPM           1495 non-null   float64       
 8   PTDVG          161 non-null    float64       
 9   RVP            140 non-null    float64       
 10  QPQS           136 non-null    float64       
 11  DRWHID         2982 non-null   object        
 12  IPP_y          2610 non-null   float64       
 13  DATE_DE_KT     2611 non-null   datetime64[ns]
 14  TDK            2611 non-null   object        
 15  TDK_proba      261

## Medical Background

In [20]:
#Import ATCD

df_ATCD = pd.read_csv('ATCD_NR.csv', sep = ";", encoding = "utf-8")
df_ATCD = df_ATCD[['IPP', 'Insuffisance rénale', 'Univentriculaire', 'Traitement influent sur la coagulation']]

df_ATCD =df_ATCD.rename(columns={'IPP': 'DRWHID_Num'})

df_ATG = df_ATCD.groupby(['DRWHID_Num']).count()

df_ATG.loc[(df_ATG['Insuffisance rénale'] < 2), 'Insuffisance rénale'] = False
df_ATG.loc[(df_ATG['Insuffisance rénale'] >= 2), 'Insuffisance rénale'] = True

df_ATG.loc[(df_ATG['Univentriculaire'] < 2), 'Univentriculaire'] = False
df_ATG.loc[(df_ATG['Univentriculaire'] >= 2), 'Univentriculaire'] = True

df_ATG.loc[(df_ATG['Traitement influent sur la coagulation'] < 2), 'Traitement influent sur la coagulation'] = False
df_ATG.loc[(df_ATG['Traitement influent sur la coagulation'] >= 2), 'Traitement influent sur la coagulation'] = True

# Merge df_KTTR_TDK & df_ATCD
KTTR_TDK_ATCD = pd.merge(KTTR_TDK, df_ATG, on='DRWHID_Num', how='left')

In [21]:
KTTR_TDK_ATCD

,DRWHID_Num,IPP_x,Date document,POIDS,SAT,SVO2,PAPS,PAPM,PTDVG,RVP,QPQS,DRWHID,IPP_y,DATE_DE_KT,TDK,TDK_proba,Insuffisance rénale,Univentriculaire,Traitement influent sur la coagulation
0,205539,8.002754e+09,2010-01-05,24.1,81.0,NaN,11.0,9.0,NaN,NaN,NaN,DRWH-205539,8.002754e+09,2010-01-05,DX,(0.9432),False,True,True
1,249116,8.003831e+09,2010-01-07,NaN,100.0,71.000000,69.0,51.0,NaN,NaN,NaN,DRWH-249116,8.003831e+09,2010-01-07,DX,(0.9221),NaN,NaN,NaN
2,65610,8.004019e+09,2010-01-11,16.9,97.0,64.500000,110.0,71.0,NaN,NaN,NaN,DRWH-65610,8.004019e+09,2010-01-11,DX,(0.8898),False,False,True
3,37634,1.406018e+09,2010-01-12,13.0,88.0,62.500000,71.0,54.0,NaN,NaN,NaN,DRWH-37634,1.406018e+09,2010-01-12,DX,(0.9086),NaN,NaN,NaN
4,32087,8.003891e+09,2010-01-14,4.0,88.0,49.000000,NaN,NaN,NaN,NaN,NaN,DRWH-32087,8.003891e+09,2010-01-14,DX,(0.9982),False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2977,376496,8.007958e+09,2016-07-01,15.0,92.0,60.333333,66.0,49.0,NaN,9.7,NaN,DRWH-376496,8.007958e+09,2016-07-01,DX,(0.9823),False,False,True
2978,377416,8.007959e+09,2016-07-01,45.0,100.0,NaN,100.0,78.0,7.0,NaN,NaN,DRWH-377416,8.007959e+09,2016-07-01,DX,(0.9807),False,False,True
2979,366552,8.006989e+09,2016-07-01,3.8,92.0,70.000000,44.0,33.0,14.0,NaN,NaN,DRWH-366552,8.006989e+09,2016-07-01,DX,(0.9998),False,True,True
2980,209201,8.003878e+09,2016-10-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DRWH-209201,NaN,NaT,NaN,NaN,False,False,True


## Biology results

Add to KTTR_TDK_ATCD biology results extracts by 'EXPORT DATA' tools in DRWH.

In [22]:
# Platelet
plq = pd.read_csv('plaquettes.csv', sep=';', encoding='utf-8', 
                  parse_dates=['DDBio'], dayfirst=True)

plq.loc[(plq['PLQ'] < 10) | (800 < plq['PLQ']), 'PLQ'] = np.nan

plq.rename(columns = {'IPP': 'DRWHID'}, inplace = True)

# Merge plaquettes and KTTR_TDK_ATCD on DRWHID
df_kta_PLQ = pd.merge(KTTR_TDK_ATCD[['DRWHID', 'Date document']],
                      plq[['DRWHID', 'DDBio', 'PLQ']],
                      how='left',
                      on='DRWHID')

# Keep only valid DDBIO
df_kta_PLQ = df_kta_PLQ[df_kta_PLQ['DDBio'].notna() & df_kta_PLQ['PLQ'].notna()]

# Keep bio near to the procedure
df_kta_PLQ['time_laps'] = abs(df_kta_PLQ['Date document'] - df_kta_PLQ['DDBio'])

df_kta_PLQ = df_kta_PLQ[(df_kta_PLQ['time_laps'] >= pd.Timedelta(0,'D')) 
                        & (df_kta_PLQ['time_laps'] <= pd.Timedelta(30,'D'))]

df_kta_PLQ = df_kta_PLQ.sort_values(['time_laps']).drop_duplicates(subset='DRWHID')

print('Nombre de patients pour qui on a des plaquettes', df_kta_PLQ['DRWHID'].nunique())

# merge
df_ktas = pd.merge(KTTR_TDK_ATCD, df_kta_PLQ[['DRWHID', 'PLQ']], how='left',on='DRWHID')

Nombre de patients pour qui on a des plaquettes 840


In [23]:
# fibrinogen
fbg = pd.read_csv('fbg.csv', sep=';', encoding='utf-8', 
                  parse_dates=['DDBio'], dayfirst=True,
                 usecols=['IPP', 'DDBio', 'FBG1'])

fbg.rename(columns = {'IPP': 'DRWHID',
                     'FBG1': 'FBG'}, inplace = True)

df_kta_FBG = pd.merge(KTTR_TDK_ATCD[['DRWHID', 'Date document']],
                      fbg[['DRWHID', 'DDBio', 'FBG']],
                      how='left',
                      on='DRWHID')

# Keep only valid DDBIO
df_kta_FBG = df_kta_FBG[df_kta_FBG['DDBio'].notna() & df_kta_FBG['FBG'].notna()]
df_kta_FBG['time_laps'] = abs(df_kta_FBG['Date document'] - df_kta_FBG['DDBio'])
df_kta_FBG = df_kta_FBG[(df_kta_FBG['time_laps'] >= pd.Timedelta(0,'D')) 
                        & (df_kta_FBG['time_laps'] <= pd.Timedelta(30,'D'))]
df_kta_FBG = df_kta_FBG.sort_values(['time_laps']).drop_duplicates(subset='DRWHID')
print('Nombre de patients pour qui on a le fibrinogene', df_kta_FBG['DRWHID'].nunique())

# merge
df_ktas = pd.merge(df_ktas, df_kta_FBG[['DRWHID', 'FBG']], how='left', on='DRWHID')

# TP
tp = pd.read_csv('tp.csv', sep=';', encoding='utf-8', 
                 parse_dates=['DDBio'], dayfirst=True,
                 usecols=['IPP', 'DDBio', 'TP'])


tp.rename(columns={'IPP': 'DRWHID'}, inplace=True)

df_kta_TP = pd.merge(KTTR_TDK_ATCD[['DRWHID', 'Date document']],
                      tp[['DRWHID', 'DDBio', 'TP']],
                      how='left',
                      on='DRWHID')

# Keep only valid DDBIO
df_kta_TP = df_kta_TP[df_kta_TP['DDBio'].notna()]
df_kta_TP = df_kta_TP[df_kta_TP['TP'].notna()]

df_kta_TP['time_laps'] = df_kta_TP['Date document'] - df_kta_TP['DDBio']


df_kta_TP = df_kta_TP[(df_kta_TP['time_laps'] >= pd.Timedelta(0,'D')) 
                        & (df_kta_TP['time_laps'] <= pd.Timedelta(30,'D'))]

df_kta_TP = df_kta_TP.sort_values(['time_laps']).drop_duplicates(subset='DRWHID')

print('Nombre de patients pour qui on a le TP', df_kta_TP['DRWHID'].nunique())

# Merge TP
df_ktas = pd.merge(df_ktas,
                   df_kta_TP[['DRWHID', 'TP']],
                      how='left',
                      on='DRWHID')

Nombre de patients pour qui on a le fibrinogene 75
Nombre de patients pour qui on a le TP 549


In [24]:
# TCA
tca = pd.read_csv('tca.csv', sep=';', encoding='utf-8', 
                  parse_dates=['DDBio'], dayfirst=True,
                 usecols=['IPP', 'DDBio', 'TCA'])
tca.rename(columns={'IPP': 'DRWHID'}, inplace = True)
df_kta_TCA = pd.merge(KTTR_TDK_ATCD[['DRWHID', 'Date document']],
                      tca[['DRWHID', 'DDBio', 'TCA']],
                      how='left',
                      on='DRWHID')

# Keep only valid DDBIO
df_kta_TCA = df_kta_TCA[df_kta_TCA['DDBio'].notna() & df_kta_TCA['TCA'].notna()]
df_kta_TCA['time_laps'] = df_kta_TCA['Date document'] - df_kta_TCA['DDBio']
df_kta_TCA = df_kta_TCA.sort_values(['time_laps']).drop_duplicates(subset='DRWHID')

print('Nombre de patients pour qui on a le TCA', df_kta_TCA['DRWHID'].nunique())

#Merge TCA
df_ktas = pd.merge(df_ktas,
                   df_kta_TCA[['DRWHID', 'TCA']],
                      how='left',
                      on='DRWHID')

Nombre de patients pour qui on a le TCA 326


In [25]:
df_ktas = df_ktas[['DRWHID_Num', 'IPP_x', 'Date document', 'POIDS', 'PTDVG', 'SAT', 'PAPS',
       'PAPM', 'RVP', 'QPQS', 'SVO2', 'DRWHID', 'TDK',
       'Insuffisance rénale', 'Univentriculaire',
       'Traitement influent sur la coagulation', 'PLQ', 'FBG', 'TP', 'TCA']]

df_ktas.rename(columns={'IPP_x': 'IPP'}, inplace=True)

In [26]:
df_ktas

,DRWHID_Num,IPP,Date document,POIDS,PTDVG,SAT,PAPS,PAPM,RVP,QPQS,SVO2,DRWHID,TDK,Insuffisance rénale,Univentriculaire,Traitement influent sur la coagulation,PLQ,FBG,TP,TCA
0,205539,8.002754e+09,2010-01-05,24.1,NaN,81.0,11.0,9.0,NaN,NaN,NaN,DRWH-205539,DX,False,True,True,NaN,NaN,NaN,1.0
1,249116,8.003831e+09,2010-01-07,NaN,NaN,100.0,69.0,51.0,NaN,NaN,71.000000,DRWH-249116,DX,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,65610,8.004019e+09,2010-01-11,16.9,NaN,97.0,110.0,71.0,NaN,NaN,64.500000,DRWH-65610,DX,False,False,True,NaN,NaN,NaN,NaN
3,37634,1.406018e+09,2010-01-12,13.0,NaN,88.0,71.0,54.0,NaN,NaN,62.500000,DRWH-37634,DX,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,32087,8.003891e+09,2010-01-14,4.0,NaN,88.0,NaN,NaN,NaN,NaN,49.000000,DRWH-32087,DX,False,True,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2977,376496,8.007958e+09,2016-07-01,15.0,NaN,92.0,66.0,49.0,9.7,NaN,60.333333,DRWH-376496,DX,False,False,True,NaN,2.3,NaN,NaN
2978,377416,8.007959e+09,2016-07-01,45.0,7.0,100.0,100.0,78.0,NaN,NaN,NaN,DRWH-377416,DX,False,False,True,78.0,3.0,NaN,NaN
2979,366552,8.006989e+09,2016-07-01,3.8,14.0,92.0,44.0,33.0,NaN,NaN,70.000000,DRWH-366552,DX,False,True,True,NaN,1.8,NaN,NaN
2980,209201,8.003878e+09,2016-10-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DRWH-209201,NaN,False,False,True,NaN,NaN,NaN,NaN


## Age on the day of the catheterization

In [27]:
ddn = pd.read_csv('DRWHNUM_BIRTH_DATE_20102016.csv', sep = ";", encoding="utf-8", 
                  parse_dates=['BIRTH_DATE'], dayfirst=True)

ddn = ddn.drop_duplicates(subset='PATIENT_NUM')

In [28]:
# Merge with df_ktas
df_ktasn = pd.merge(
    df_ktas,
    ddn,
    left_on='DRWHID_Num',
    right_on='PATIENT_NUM',
    how='left'              )

In [29]:
df_ktasn

,DRWHID_Num,IPP,Date document,POIDS,PTDVG,SAT,PAPS,PAPM,RVP,QPQS,...,TDK,Insuffisance rénale,Univentriculaire,Traitement influent sur la coagulation,PLQ,FBG,TP,TCA,PATIENT_NUM,BIRTH_DATE
0,205539,8.002754e+09,2010-01-05,24.1,NaN,81.0,11.0,9.0,NaN,NaN,...,DX,False,True,True,NaN,NaN,NaN,1.0,205539,2003-01-22
1,249116,8.003831e+09,2010-01-07,NaN,NaN,100.0,69.0,51.0,NaN,NaN,...,DX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,249116,1996-01-26
2,65610,8.004019e+09,2010-01-11,16.9,NaN,97.0,110.0,71.0,NaN,NaN,...,DX,False,False,True,NaN,NaN,NaN,NaN,65610,2005-06-14
3,37634,1.406018e+09,2010-01-12,13.0,NaN,88.0,71.0,54.0,NaN,NaN,...,DX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37634,2006-03-04
4,32087,8.003891e+09,2010-01-14,4.0,NaN,88.0,NaN,NaN,NaN,NaN,...,DX,False,True,True,NaN,NaN,NaN,NaN,32087,2009-11-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2977,376496,8.007958e+09,2016-07-01,15.0,NaN,92.0,66.0,49.0,9.7,NaN,...,DX,False,False,True,NaN,2.3,NaN,NaN,376496,2011-12-03
2978,377416,8.007959e+09,2016-07-01,45.0,7.0,100.0,100.0,78.0,NaN,NaN,...,DX,False,False,True,78.0,3.0,NaN,NaN,377416,2002-03-20
2979,366552,8.006989e+09,2016-07-01,3.8,14.0,92.0,44.0,33.0,NaN,NaN,...,DX,False,True,True,NaN,1.8,NaN,NaN,366552,2016-02-23
2980,209201,8.003878e+09,2016-10-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,False,False,True,NaN,NaN,NaN,NaN,209201,2000-11-14


In [30]:
# Calculate the age on the day of the catheterization
df_ktasn['KT_AGE'] = df_ktasn['Date document'] - df_ktasn['BIRTH_DATE']

# Remove aberrant ages 
df_ktasn = df_ktasn.loc[~(df_ktasn.KT_AGE < dt.timedelta(days=0))]
print('Nombre de patients inclus dans l étude:', df_ktasn['DRWHID'].nunique())

Nombre de patients inclus dans l étude: 2980


## Adverse events

In [31]:
## Matching the complication table with the correspondence table

# correspondence table
tbl_cf = pd.read_csv('table_de_correspondance.csv', 
                     sep=";", 
                     encoding="utf-8", 
                     index_col=0)
# Result table (unlab_classifier_exemples_positifs_verifie is the table where 
# I checked every positive complications)
df_RESULT = pd.read_csv('unlab_classifier_exemple_positifs_verifies.csv', 
                        sep=";", encoding="utf-8")
df_RESULT['index'] = df_RESULT['index'].str.extract('(\d+)')

# add train, test, dev result
df_train = pd.read_csv('train.csv', 
                        sep=";", encoding="utf-8",
                      usecols=['index', 'label'])
df_train['index'] = df_train['index'].str.extract('(\d+)')
df_train.rename(columns={'label': 'result'}, inplace=True)

df_test = pd.read_csv('test.csv', sep=";", encoding="utf-8",
                      usecols=['index', 'label'])
df_test['index'] = df_test['index'].str.extract('(\d+)')
df_test.rename(columns={'label': 'result'}, inplace=True)

df_dev = pd.read_csv('dev.csv', sep=";", encoding="utf-8",
                      usecols=['index', 'label'])
df_dev['index'] = df_dev['index'].str.extract('(\d+)')
df_dev.rename(columns={'label': 'result'}, inplace=True)

df_RESULT = pd.concat([df_RESULT, df_train, df_test, df_dev])

# df_RESULT.to_csv('result_test.csv', sep=';', encoding='utf-8' )

# set index as int64
df_RESULT = df_RESULT.astype({'index': 'int64'})
df_RESULT = df_RESULT.set_index('index')

# merge
df_merge = tbl_cf.merge(df_RESULT, left_index=True, right_index=True)

print(df_merge.groupby('type_complication')['result'].mean().sort_values(ascending=False))

df_merge.rename(columns={'IPP': 'DRWHID'}, inplace=True)
df_RESULT = df_merge[['DRWHID', 'result', 'DDK', 'texte_complication']].dropna()
df_RESULT.loc[:,'result'] = df_RESULT['result'].astype(int)
# df_RESULT = df_RESULT[['DRWHID', 'result']]
df_RESULT = df_RESULT.groupby('DRWHID').sum()
print(df_RESULT['result'].value_counts())

type_complication
CHOC_ANA          0.357143
LESION_VALVE      0.264706
ACR               0.253550
AVC               0.141753
EMBOLIE_PTH       0.103448
HEMOPTYSIE        0.100977
DISSECTION        0.087515
TDR               0.077563
CHOC_HD           0.066448
TAMPONNADE        0.064000
ISCH              0.053333
DC                0.037249
IOT               0.034740
REINTERVENTION    0.032103
ECMO              0.023881
SAIGNEMENT        0.023379
DIALYSE           0.020231
TRANSFUSION       0.014789
EMBOLIE_GAZ       0.012195
PNO               0.000000
Name: result, dtype: float64
0     1929
1      172
2       80
3       29
4       12
6       11
5        7
8        6
7        6
10       5
9        4
11       3
21       2
17       1
36       1
18       1
20       1
13       1
39       1
Name: result, dtype: int64


In [32]:
df_ktasnr = pd.merge(df_ktasn,
                     df_RESULT,
                     left_on='DRWHID',
                     right_on='DRWHID',
                     how='left')


print('Nombre de patients:')
print(df_ktasnr['DRWHID'].nunique())

print('------------------')
print(df_ktasnr['result'].value_counts())
#print(df_ktasnr.groupby(df_ktasnr["Date document"].dt.year).count().plot(kind="bar", legend=None))
#print()

Nombre de patients:
2980
------------------
0.0     1121
1.0      103
2.0       51
3.0       18
6.0        9
4.0        7
8.0        5
7.0        4
10.0       3
9.0        2
5.0        2
21.0       2
18.0       1
36.0       1
20.0       1
13.0       1
39.0       1
Name: result, dtype: int64


In [33]:
# ktasnr is the final result of our automatically extracted protocol
# ktasnr.csv will be used to describe missing data, and patient characteristics

df_ktasnr['result'][df_ktasnr['result'] > 0] = 1
df_ktasnr.to_csv(f'{datefile}_ktasnr.csv', sep=';', encoding='utf-8')
df_ktasnr['result'].value_counts()

# to save datas before replace missing values: 
data_extract = df_ktasnr

/home/sophie/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [34]:
print(df_ktasnr['result'].value_counts())

0.0    1121
1.0     211
Name: result, dtype: int64


In [35]:
df_ktasnr.groupby('result').agg(['mean', 'std'])

DRWHID_Num                         IPP                    POIDS  \
                mean           std          mean           std       mean   
result                                                                      
0.0     2.141307e+06  8.597542e+06  6.657869e+09  2.579698e+09  17.909947   
1.0     2.260279e+06  8.847235e+06  6.530442e+09  2.696157e+09  16.129722   

                       PTDVG                   SAT            ...         PLQ  \
              std       mean        std       mean       std  ...        mean   
result                                                        ...               
0.0     17.879755  13.426117   5.438948  91.884615  8.086908  ...  266.339869   
1.0     16.966794  15.727273  13.602139  92.883212  7.814076  ...  264.208955   

                         FBG                   TP                  TCA  \
               std      mean       std       mean        std      mean   
result                                                                   
0.0     107.028978  2.635185  1.231376  83.415301  13.879473  1.055248   
1.0     101.737413  2.033333  0.640312  81.452381  13.581196  1.043333   

                   PATIENT_NUM                
             std          mean           std  
result                                        
0.0     0.182847  2.141307e+06  8.597542e+06  
1.0     0.139895  2.260279e+06  8.847235e+06  

[2 rows x 30 columns]

In [36]:
corr = df_ktasnr.corr()

corr.style.background_gradient(cmap='coolwarm')

,DRWHID_Num,IPP,POIDS,PTDVG,SAT,PAPS,PAPM,RVP,QPQS,SVO2,PLQ,FBG,TP,TCA,PATIENT_NUM,result
DRWHID_Num,1.000000,-0.338726,-0.022074,-0.061184,-0.022064,0.027593,0.025366,-0.044402,-0.004930,0.028931,-0.003468,-0.121604,0.005557,-0.038278,1.000000,0.005033
IPP,-0.338726,1.000000,-0.038802,-0.009823,-0.050767,-0.009569,-0.044082,0.039693,0.015538,0.095502,-0.014329,-0.060797,-0.010852,0.007611,-0.338726,-0.017886
POIDS,-0.022074,-0.038802,1.000000,0.128085,0.280028,-0.093171,-0.123617,0.277218,-0.051136,0.121612,-0.247903,0.383409,-0.085962,-0.128608,-0.022074,-0.036749
PTDVG,-0.061184,-0.009823,0.128085,1.000000,0.166688,0.089889,0.080414,-0.394021,0.273943,0.223082,-0.242986,-0.286006,0.109218,-0.128702,-0.061184,0.105031
SAT,-0.022064,-0.050767,0.280028,0.166688,1.000000,-0.185578,-0.099116,-0.358291,0.160456,0.392241,0.125613,0.175405,0.230498,-0.093075,-0.022064,0.045610
PAPS,0.027593,-0.009569,-0.093171,0.089889,-0.185578,1.000000,0.972090,0.698183,0.107319,-0.238481,-0.140015,-0.002663,-0.121130,0.000620,0.027593,0.045667
PAPM,0.025366,-0.044082,-0.123617,0.080414,-0.099116,0.972090,1.000000,0.699978,0.075707,-0.208852,-0.105315,0.104186,-0.042715,0.012057,0.025366,0.044616
RVP,-0.044402,0.039693,0.277218,-0.394021,-0.358291,0.698183,0.699978,1.000000,-0.148607,-0.286200,-0.259644,-0.099823,0.015560,0.350481,-0.044402,0.077699
QPQS,-0.004930,0.015538,-0.051136,0.273943,0.160456,0.107319,0.075707,-0.148607,1.000000,-0.034971,-0.039789,0.052775,-0.018501,0.204575,-0.004930,0.048019
SVO2,0.028931,0.095502,0.121612,0.223082,0.392241,-0.238481,-0.208852,-0.286200,-0.034971,1.000000,0.083917,-0.308163,0.184334,0.082030,0.028931,-0.067418


## Replace missing data with normal values

In [37]:
# we replace missing weights by a weight that will not influence the score (7 kg)

df_ktasnr.loc[:,'POIDS'] = df_ktasnr['POIDS'].fillna(7)

# same thing for the cath type: Cathétérisme diagnostique does not influence the score

df_ktasnr.loc[:,'TDK'] = df_ktasnr['TDK'].fillna('DX')

In [38]:
# if there is no mention in the text, 
# we considered that the patient does not have the antecedent

df_ktasnr = df_ktasnr.fillna({'Insuffisance rénale':False, 
                              'Univentriculaire' :False, 
                              'Traitement influent sur la coagulation' : False})

In [39]:
print('Univentriculaire:')
print(df_ktasnr['Univentriculaire'].value_counts())

print('Insufisants rénaux:')
print(df_ktasnr['Insuffisance rénale'].value_counts())

print('Anticoagulés:')
print(df_ktasnr['Traitement influent sur la coagulation'].value_counts())

Univentriculaire:
False    2627
True      353
Name: Univentriculaire, dtype: int64
Insufisants rénaux:
False    2957
True       23
Name: Insuffisance rénale, dtype: int64
Anticoagulés:
False    2415
True      565
Name: Traitement influent sur la coagulation, dtype: int64


In [40]:
# Missing biology data are replace with normal values

df_ktasnr.loc[:,'PLQ'] = df_ktasnr['PLQ'].fillna(250)
df_ktasnr.loc[:,'TCA'] = df_ktasnr['TCA'].fillna(1)
df_ktasnr.loc[:,'TP'] = df_ktasnr['TP'].fillna(80)
df_ktasnr.loc[:,'FBG'] = df_ktasnr['FBG'].fillna(2)

In [41]:
# if there is no mention in the text, 
# we considered that the patient did not undergone MAE

df_ktasnr.loc[:,'result'] = df_ktasnr['result'].fillna(0)

patient_mae = (df_ktasnr['result'].value_counts()[1])

print('Nombre de patients ayant fait au moins un événement indésirable:')
print(patient_mae)

print('Pourcentage de patients ayant fait au moins un événement indésirable:')
print(patient_mae/retenus*100)

Nombre de patients ayant fait au moins un événement indésirable:
211
Pourcentage de patients ayant fait au moins un événement indésirable:
7.0757880617035545


In [42]:
# we replace missing age by 365 days because that will not influence the score

df_ktasnr.loc[:,'KT_AGE'] = df_ktasnr['KT_AGE'].fillna(pd.Timedelta(value=366, unit='d'))

In [43]:
df_ktasnr.loc[:,'KT_AGE'] = df_ktasnr['KT_AGE'].dt.days

In [44]:
df_ktasnr['KT_AGE']

0       2540
1       5095
2       1672
3       1410
4         72
        ... 
2975    1672
2976    5217
2977     129
2978    5816
2979    5875
Name: KT_AGE, Length: 2980, dtype: int64

In [45]:
df_ktasnr.to_csv(f'{datefile}_score.csv', sep=';', encoding='utf-8')